## Выбираем лучшие настройки для тикеров

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
TOKEN = os.getenv("INVEST_TOKEN")

In [ ]:
from dto.config_dto import ConfigDTO
from lib.task_progress import TaskProgress
from test_env.test_alg import TestAlgorithm
from concurrent.futures import ThreadPoolExecutor, as_completed

test_configs = [
    (ConfigDTO(
        ticker=t_config.ticker,
        step_max_cnt=max_shares,
        step_base_cnt=base_shares,
        step_lots=1,

        majority_trade=t_config.majority_trade,
        maj_to_zero=t_config.maj_to_zero,
        pretest_period=pretest_period,

        threshold_buy_steps=0,
        threshold_sell_steps=0,
        stop_up_p=0,
        stop_down_p=0,

        step_size=t_config.step_size,
        step_set_orders_cnt=step_cnt,
    ))
    for t_config in [
        # ConfigDTO.from_repr_string('DELI 3/-3/2 x l2 x 1.6¤, |s0 b0| |u0.0 d0.0| maj+z+'),
        # ConfigDTO.from_repr_string('ETLN 3/0/2 x l5 x 1.2¤, |s0 b0| |u0.0 d0.0| maj-z-'),
        # ConfigDTO.from_repr_string('EUTR 3/-3/2 x l2 x 1.2¤, |s0 b0| |u0.0 d0.0| maj+z+'),
        ConfigDTO.from_repr_string('MTLR 3/-3/3 x l2 x 1.2¤, |s0 b0| |u0.0 d0.0| maj+z+'),
        ConfigDTO.from_repr_string('POLY 3/3/2 x l2 x 1.0¤, |s0 b0| |u0.0 d0.0| maj-z-'),
        # ConfigDTO.from_repr_string('QIWI 3/3/2 x l2 x 1.2¤, |s0 b0| |u0.0 d0.0| maj-z-'),
        # ConfigDTO.from_repr_string('RNFT 3/3/3 x l2 x 1.2¤, |s0 b0| |u0.0 d0.0| maj+z+'),
        # ConfigDTO.from_repr_string('SPBE 3/0/3 x l5 x 1.2¤, |s0 b0| |u0.0 d0.0| maj-z-'),
        # ConfigDTO.from_repr_string('UWGN 4/0/3 x l7 x 1.4¤, |s0 b0| |u0.0 d0.0| maj-z-'),
    ]
    for max_shares in [3]
    for base_shares in [0]
    for step_size_shift in [0]
    for step_cnt in [3]  # вот тут некоторым удобнее в 2. можно потестить в вариаторе 2 и 3
    for pretest_period in range(3) # todo
]


def run_test(config: ConfigDTO):
    test_alg = TestAlgorithm(TOKEN, do_printing=False, config=config)
    return test_alg.test(
        last_test_date='2024-05-01',
        test_days_num=7,  # todo
        shares_count=0,

        auto_conf_days_freq=1,
        auto_conf_prev_days=config.pretest_period,
    )

results = []
progress = TaskProgress(len(test_configs))

with ThreadPoolExecutor(max_workers=4) as executor:
    future_to_params = {executor.submit(run_test, config): config for config in test_configs}
    
    for future in as_completed(future_to_params):
        params = future_to_params[future]
        res = future.result()
        if res:
            results.append(res)
        progress.update_progress()

# Вывод результатов или их дальнейшая обработка
sorted_results = sorted(results, key=lambda x: (x['config'].ticker, -float(x['profit_p'])), reverse=False)

print()
for item in sorted_results:
    print(item)

Выбираем лучшие настройки
---




#82 Динамическая подстройка параметров запуска
---

исходный вариант алгоритма
{'profit': 549.7, 'profit_p': 55.41, 'config': RNFT 4/pre13:0/3 x l2 x 1.2¤, |s0 b0| |u0.0 d0.0| maj+z+ , 'op_cnt': 2318}

на pre
{'profit': 253.8, 'profit_p': 51.17, 'config': RNFT 4/pre13:0/3 x l1 x 1.2¤, |s0 b0| |u0.0 d0.0| maj+z+ , 'op_cnt': 2318}

last_test_date='2024-05-01',
test_days_num=365,  # 365,
auto_conf_days_freq=7,
auto_conf_prev_days=7,
{'profit': 42.5, 'profit_p': 8.57, 'config': RNFT 4/0/3 x l1 x 1.2¤, |s0 b0| |u0.0 d0.0| maj+z+ , 'op_cnt': 2283}



